```
Langchain 을 활용한 모델 사용,비용 모니터링 및 캐싱 전략
    GPT-4o-mini  GPT-3.5-Turbo 비용이 60% 저렴
    LangChain V0.3x 부터 openAi가 별도 패키지로 분리 필요 패키지를 설치 langchain-openai 필요 
    토큰사용량 추적, 캐싱을 위한 langchain-community도 별도설치
    환경변수 변수 관리 패키지 python-dotevn
```


In [2]:
%pip install langchain-openai langchain-community python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.7)  # dotenv 를 하면 자동으로 OPENAI_API_KEY 참조
prompt = 'LangChain에 대해 한 문장으로 설명해줘'
result = llm.invoke(prompt)
result.content

'LangChain은 다양한 언어 모델과 데이터 소스를 연결하여 자연어 처리 애플리케이션을 구축할 수 있도록 돕는 프레임워크입니다.'

In [5]:
#사용량
result.usage_metadata

{'input_tokens': 18,
 'output_tokens': 35,
 'total_tokens': 53,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [6]:
# 콜벡함수를 통한 누적 토큰 추적(get_openai_callback)
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫번째 호출
    res1 = llm.invoke('서울의 오늘 날씨는 어떤지 알려줘?')
    print('응답1', res1.content[:10],'...')
    # 두번째 호출
    res2 = llm.invoke('파이썬으로 랭체인 사용법을 알려줘')
    print('응답2', res2.content[:10],'...')

# 누적 토큰 사용량 출력  콜백 cb에는 블록 내 전체 토큰 사용량이 누적
# 총 토큰수
print('총 토큰수:', cb.total_tokens)
# 프롬프트 토큰수
print('프롬프트 토큰수:', cb.prompt_tokens)
# 응답 토큰수
print('응답 토큰수:', cb.completion_tokens)
# 비용 계산
print('비용(USD):', cb.total_cost)


응답1 죄송하지만, 실시간 ...
응답2 LangChain은 ...
총 토큰수: 737
프롬프트 토큰수: 39
응답 토큰수: 698
비용(USD): 0.00042464999999999994


In [ ]:
# LangChain의 LLM 응답캐싱 (InMemory Cache, SQLiteCache)
# 동일한 질문은 저장해 뒀다가 응답에 사용
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCache 설정
set_llm_cache(InMemoryCache()) # 캐시 기능 키기 # 블로그 

In [ ]:
# 캐시 사용 전후를 비교, 같은 질문을 두번 호출
query = "재미있는 유머 하나 알려줘"
# 첫번째 호출(캐시에 없으면 api 호출 발생)
result1 = llm.invoke(query)
print("응답1:", result1.content)
print('#'*100)
# 두번째 호출(동일한 query, 캐시를 확인하고 동일 질문이면 api 미호출)
result2 = llm.invoke(query) # query가 한글자라도 다르면 캐시 미사용한다 # 블로그 
print("응답2:", result2.content)


응답1: 물론이죠! 다음과 같은 유머는 어떨까요?

왜 컴퓨터는 바다에 빠지지 않을까요?

왜냐하면 네트워크가 있기 때문이죠! 

(이 유머는 '네트워크'와 '물'의 연결을 이용한 말장난입니다!)
####################################################################################################
응답2: 물론이죠! 다음과 같은 유머는 어떨까요?

왜 컴퓨터는 바다에 빠지지 않을까요?

왜냐하면 네트워크가 있기 때문이죠! 

(이 유머는 '네트워크'와 '물'의 연결을 이용한 말장난입니다!)


In [ ]:
# 실행시간 측정
import time
# 첫 번째 호출 시간
query = "점심메뉴 추천해줘"
start = time.time();llm.invoke(query);end = time.time()
print(f"첫 번째 호출 시간: {end - start}")

start = time.time();llm.invoke(query);end = time.time()
print(f"두 번째 호출 시간: {end - start}")

첫 번째 호출 시간: 4.35541844367981
두 번째 호출 시간: 0.0


In [ ]:
# SQLiteCache 사용(디스크기반 캐시)
import os
from langchain_core.caches import SQLiteCache
# 기존 캐시 DB 파일이 있다면 삭제(.langchain.db 초기화)
if os.path.exists('.langchain.db'):
    os.remove('.langchain.db')

# SQLiteCache 캐시 설정(지정한 경로의 DB 파일을 생성 / 사용)
set_llm_cache(SQLiteCache(database_path='.langchain.db')) # .langchain.db있으면 사용 없으면 생성